# Сборный проект
## Постановка задачи

Заказчик исследования - каршеринговая компания. Задача - создать систему, которая могла бы оценить риск ДТП по выбранному маршруту движения. Под риском понимается вероятность ДТП с любым повреждением транспортного средства. Как только водитель забронировал автомобиль, сел за руль и выбрал маршрут, система должна оценить уровень риска. Если уровень риска высок, водитель увидит предупреждение и рекомендации по маршруту.


Идея создания такой системы находится в стадии предварительного обсуждения и проработки. Чёткого алгоритма работы и подобных решений на рынке ещё не существует. Текущая задача — понять, возможно ли предсказывать ДТП, опираясь на исторические данные одного из регионов.

Идея решения задачи от заказчика: 
* Создать модель предсказания ДТП (целевое значение — at_fault (виновник) в таблице parties)
* Для модели выбрать тип виновника — только машина (car)
* Выбрать случаи, когда ДТП привело к любым повреждениям транспортного средства, кроме типа SCRATCH (царапина)
* Для моделирования ограничиться данными за 2012 год — они самые свежие
* Обязательное условие — учесть фактор возраста автомобиля
* На основе модели исследовать основные факторы ДТП
* Понять, помогут ли результаты моделирования и анализ важности факторов ответить на вопросы:
** Возможно ли создать адекватную системы оценки водительского риска при выдаче авто?
** Какие ещё факторы нужно учесть?
** Нужно ли оборудовать автомобиль какими-либо датчиками или камерой?

Для решения поставленной задачи предполагаются следующие действия:
* изучить исходные данные, сделав выгрузку SQL таблиц
* написать список задач для дополнительного исследования
* отобрать факторы для обучения моделей
* создать датасет для обучения моделей
* обучить на датасете три модели
* выбрать лучшую из них и проверить ее в работе
* проанализировать важность факторов ДТП и провести по ним дополнительное исследование
* составить рекомендации для компании-заказчика


## Импорт библиотек

In [1]:
!pip install scikit-learn==1.1.3

In [2]:
import math # standard
from math import ceil # standard
from pathlib import Path # standard
import random # standard
import re # standard
import warnings # standard library
warnings.filterwarnings('ignore')

from catboost import CatBoostClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)
from scipy import stats
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sqlalchemy import create_engine 

## Импорт и первичное исследование данных
### Создание подключения

In [3]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-science-vehicle-db' # название базы данных
}  

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [4]:
engine = create_engine(connection_string) 

### Краткое описание таблиц

* collisions — общая информация о ДТП

Имеет уникальный case_id. Эта таблица описывает общую информацию о ДТП. Например, где оно произошло и когда.

* parties — информация об участниках ДТП

Имеет неуникальный case_id, который сопоставляется с соответствующим ДТП в таблице collisions. Каждая строка здесь описывает одну из сторон, участвующих в ДТП. Если столкнулись две машины, в этой таблице должно быть две строки с совпадением case_id. Если нужен уникальный идентификатор, это case_id and party_number.

* vehicles — информация о пострадавших машинах

Имеет неуникальные case_id и неуникальные party_number, которые сопоставляются с таблицей collisions и таблицей parties. Если нужен уникальный идентификатор, это case_id and party_number.

### Таблица collisions
Таблица collisions содержит информацию о транспортных происшествиях. Первичный ключ - `case_id`. В таблице 1_400_000 записей. 

#### Описание данных
Признаки, содержащиеся в таблице:
* 'case_id' - уникальный id происшествия
* 'county_city_location' - место происшествия
* 'county_location' - тоже место происшествия
* 'distance' - расстояние от главной дороги
* 'direction' - направление движения
* 'intersection' - булево поле, отвечающее на вопрос: "на перекрестке?"
* 'weather_1' - погода
* 'location_type' - тип места происшествия (трасса, перекресток и тд)
* 'collision_damage' - степень серьезности ДТП
* 'party_count' - количество сторон ДТП
* 'primary_collision_factor' - главный фактор ДТП
* 'pcf_violation_category' - категория нарушения
* 'type_of_collision' - тип происшествия
* 'motor_vehicle_involved_with' - второе транспортное средство, участвующее в ДТП
* 'road_surface' - поверхность дороги (сухая, скользкая, мокрая...)
* 'road_condition_1' - состояние дороги (нормальное, с ямами, ремонт, отсутствие дорожного покрытия и тд)
* 'lighting' - освещенность
* 'control_device' - устройство управление (светофор?)
* 'collision_date' - дата происшествия 
* 'collision_time' - время происшествия

#### Типы данных
Все колонки, кроме `distance`, `party_count`, `intersection` имеют тип object. В некоторых случаях типы данных не соответствуют факту, возможно стоит конвертировать:
* `intersection` ->  boolean
* `collision_date` -> date
* `collision_time` -> time
* `case_id` -> int

#### Пустые значения
В таблице есть пустые значения.

In [5]:
def overall_info(df):
    print('Shape:')
    print(df.shape)
    print('Head:')
    display(df.head())
    print('Describe df:')
    display(df.describe())
    print(); print()
    print(df.info())
    print(); print()
    print('Nan values:')
    display(df.isna().mean())
    print('Distributions of numerical values:')
    df.hist(figsize=(27, 9))

In [6]:
def print_uniq_values_frequency(df):
    print('Уникальные значения, %')
    for column in df.columns:
        if column not in ['id', 'case_id']:
            print(column); print()
            print(df[column].value_counts() / df.shape[0] * 100)
            print(); print()

In [7]:
project_path = Path('broken/home/klarazetkin/Documents/yandex/module_4/project_5')
              
def save_df(df, project_path, df_file_name):
    if project_path.is_dir():
        file_name = str(project_path) + '/' + df_file_name   
        file_path = Path(file_name) 
        file_path.parent.mkdir(parents=True, exist_ok=True)  
        df.to_csv(file_path)  
    else:
        print("Local backup failed, you probably don't need it")

### Таблица parties
#### Описание данных
Признаки, содержащиеся в таблице:
* id - уникальный id стороны ДТП
* case_id - уникальный id происшествия (вторичный ключ)
* party_number - номер стороны-участника ДТП
* party_type - тип участника (машина, пешеход, дерево...)
* at_fault - виновность стороны в ДТП
* insurance_premium - сумма страховки
* party_sobriety - трезвость участника ДТП
* party_drug_physical - информация о наркотическом опьянении и физическом состоянии участника
* cellphone_in_use - наличие телефона в автомобиле

#### Типы данных
Нужно будет конвертировать:
* cellphone_in_use -> boolean
* at_fault -> boolean

#### Пустые значения
В таблице есть пустые значения:
* insurance_premium     0.15
* party_sobriety        0.03
* party_drug_physical   0.84
* cellphone_in_use      0.19


### Таблица vehicles


#### Описание данных
Поля таблицы:
* id - уникальный id транспортного средства
* case_id - уникальный id происшествия из таблицы collisions
* party_number - номер стороны ДТП
* vehicle_type - тип транспортного средства
* vehicle_transmission - тип коробки передач
* vehicle_age - возраст транспортного средства

#### Типы данных
Типы данных в основном адекватны.

#### Пустые значения
В таблице есть пропуски:
* vehicle_transmission:   0.02 %
* vehicle_age:            0.02 %

### Связь таблиц

Таблицы связаны следующим образом:
* в таблице collisions есть поле case_id - это первичный ключ таблицы collisions и он соответствует уникальному идентификатору происшествия;
* по этому полю (case_id) с таблицей collisions связаны таблицы parties и vehicles - в каждой из них тоже есть поле case_id, для них это вторичный ключ;
* между собой таблицы parties и vehicles связаны по двум полям: case_id и party_number. Поле party_number означает номер стороны конфликта в данном конфликте. Соответственно, о каждом транспортном средстве мы знаем, в какое происшествие (определяемое case_id) оно попало и какой по номеру стороне конфликта (это определяется полем party_number) оно принадлежит. Про каждую сторону происшествия знаем ее номер в конфликте (party_number) и происшествие, в котором эта сторона участвовала.

Поля case_id и party_number являются вторичными ключами для таблиц parties и vehicles. Первичные ключи для этих таблиц - это id, но связь между таблицами не по id, а по party_number + case_id.

Если одно и то же транспортное средство участвовало во многих происшествиях, вероятно, оно попадет в таблицу vehicles многократно, с разными id и с разными комбинациями party_number + case_id. То есть в рамках таблицы это будут разные объекты.

Точно так же если один и тот же человек попал во многие дорожные происшествия, то он будет иметь по одной записи на происшествие в таблице parties, потому что для каждого происшествия он выступает как отдельный актор.

Проверим, что связь работает и что нет "потерявшихся" записей, когда запись о происшествии не имеет соответствующих записей в parties или vehicles. Сначала заджойним таблицы попарно.

Для всех записей collisions есть записи о сторонах ДТП: количество уникальных записей в каждой из таблиц после INNER JOIN не изменилось.

Информация по ТС есть не для всех collisions.case_id, но для некоторых collisions.case_id есть несколько ТС участников:

Логично, что при INNER JOIN выпадают записи о сторонах ДТП, для которых нет соответствующих записей о ТС.

Заджойним три таблицы и проверим корректность джойна. После итогового джойна общее количество записей не может быть больше, чем количество parties.id, потому что:
* нужно, чтоб в итоговую таблицу вошли все записи collisions
* одной записи collisions.case_id соответствует несколько записей в таблице parties, соответственно, записей в итоговой таблице будет больше, чем в collisions, число должно быть равно COUNT(parties.id), поскольку parties без соответствующих collisions нет
* транспортные средства интересны с т.з. их принадлежности сторонам конфликта, поэтому джойним их через таблицу parties по двум полям, иначе получится произведение всех записей vehicles на все записи parties при одинаковом case_id

Этот джойн и будем использовать в дальнейшем.

Как видно из итогового запроса, отсутствует информация по ТС примерно для по 43.4% случаев, причем практически все записи соответствуют ДТП с малым ущербом: из 845771 ДТП записи по сторонам ДТП и транспортным средствам есть в 238417 случаях, то есть потерялось примерно 72,8% данных.

### Ошибочные данные, выявленные на этапе ознакомления
#### Ошибочные данные в таблице parties
В таблице точно есть ошибочные данные, которые придется исправить или удалить. Например, в качестве виновной стороны ДТП видим дорожные знаки, деревья, здания. Более того, они управляют автомобилем, т.к. есть связь с таблицей vehicles.

Исследуем эти кейсы подробней.

Подозрительные данные:

В таблице записи о 88_857 участниках ДТП, которые содержат данные не о людях, а о неодушевленных предметах, это нормально. А вот 38_481 записей о неодушевленных предметах, которые стали виновниками ДТП, - это явная ошибка. Данных о пешеходах в колонке нет, или они скрыты в категории 'other'. Поскольку в работе нужно анализировать поведение водителей, все типы parties.party_type придется дропнуть. Так эта ошибка в данных будет устранена.

## Статистический анализ факторов ДТП
### В какие месяцы происходило больше всего ДТП?
#### Анализ данных без разбивки по годам
Создадим SQL запрос, чтоб ответить на этот вопрос.

In [8]:
def print_barplot(df, column_x, column_y, title='MyBarplot', figsize=(9, 6)):
    f, ax = plt.subplots(figsize=figsize)
    sns.barplot(data=df,  x=column_x, y=column_y, ci='sd', palette='viridis', ax=ax)
    plt.title(title)
    plt.xticks(rotation=90)
    plt.show()

Можно сделать предварительный вывод, что наибольшее количество ДТП случается с января по май, причем наиболее неблагополучный месяц - март, второй по опасности - январь. 

#### Проверка гипотезы о неполноте данных
Чтобы убедиться в правильности предварительного вывода проверим гипотезу о неполноте данных: возможно, в базе отсутствуют данные за месяцы каких-то годов.

Из графика видно, что начиная с июня 2012 года данные по авариям крайне неполные, а за некоторые месяцы и вовсе отсутствуют.

Явно выраженной динамики по месяцам нет, хотя можно отметить, что в октябре аварий сравнительно больше, чем в другие месяцы; а в январе-феврале обычно наблюдается спад. Поскольку модели предполагается учить на данных за 2012 год, то дату и привязанную к ней сезонность имеет смысл исключить из датасета, так как данные есть только за первые пять месяцев. 

Первоначальный вывод о том, что есть условно опасный (с января по май) и условно безопасный (с июня по декабрь) сезоны, неверен.

### Задачи на исследование
Для понимания данных целесообразно исследовать следующие вопросы:
* типичное количество участников ДТП и их самые популярные комбинации (транспортное средство, дерево, дорожный знак и тд);
* проверить гипотезу о том, что пьяные и под веществами водители чаще попадают в ДТП, где они являются единственным участником;
* проверить гипотезу о том, что пьяные и под веществами водители чаще становятся виновниками смертельных ДТП;
* зависимость количеств ДТП от состояния дорожного покрытия;
* зависимость количеств ДТП от погоде;
* зависимость количеств ДТП от времени суток и освещенности;
* распределение ДТП по наличию алкоголя в крови водителя;
* распределение ДТП по наличию наркотиков в крови водителя;
* зависимость от возраста транспортного средства;
* исследовать типичные обстоятельства ДТП с тяжелыми последствиями;
* исследовать типичные признаки водителя, винового в ДТП;
* исследовать типы транспортных средств, которые чаще всего попадают в ДТП;
* исследовать типы транспортных средств, которые чаще всего управляются виновником ДТП

#### Решение задачи 1
Опишем решение этой задачи:
* определить типы транспортных средств, которые чаще всего управляются виновником ДТП

Нужно составить SQL  запрос, связав таблицы vehicles и parties по полям `vehicles.party_number = parties.party_number AND vehicles.case_id = parties.case_id`, сгруппировать по полям parties.party_type, vehicles.vehicle_type и выбрать уникальные комбинации.

На этом этапе выяснится, что в данных есть ошибки. От коллеги ожидается, что он укажет на необходимость исправить данные перед обучением модели, а запрос исправит, чтоб он стал информативным, примерно так:

Чтоб сделать вывод, посмотрим, на чем вообще ездили водители-участники ДТП (для которых 'party_type' = 'car') безотносительно их виновности.

Можно сделать вывод о том, что никакой явной зависимости риска стать виновником ДТП от типа транспортного средства нет. Водители, ставшие виновниками ДТП, ездят на том же, на чем остальные участники ДТП.

#### Решение задачи 2
Опишем предполагаемый путь решения второй задачи:
* типичные обстоятельства ДТП с тяжелыми последствиями

Нужно составить SQL запрос: 
1. корректно связать таблицы collisions, parties и vehicles
2. сгруппировать по полям parties.party_type, vehicles.vehicle_type и выбрать уникальные комбинации
3. какие комбинации нужно выбрать, выяснится в процессе. Задачу нужно выполнить творчески.

Предварительно стоит ознакомиться с данными в таблицах, чтоб получить содержательный результат, сгруппировав значения в колонках там, где это уместно. 

Из таблицы видно, что самый частый сценарий ДТП с тяжелыми и фатальными последствиями - это трезвый водитель в ясную погоду на сухой дороге, нарушающий правила дорожного движения. В совокупности эти случаи составляют более половины случаев (55,69%).

Cгруппируем данные для красивой визуализации. Как выяснилось выше, сезон и тип ТС нас не интересуют. Колонка `location_type` в данном случае тоже не информативна.

Как видно из графика выше, самые популярные сценарии тяжелых и фатальных ДТП следующие:
* 45% - ДТП с участием 2 и более водителей, трезвых, не под наркотиками, в хорошую погоду, на сухой дороге, причина - нарушени ПДД;
    * +11% - то же самое, но в ДТП один участник;
    * +4.5% - то же самое, два участника ДТП, все трезвые, но погода облачная;
* 9% - пьяный водитель в хорошую погоду, на сухой дороге нарушает правила ПДД и попадает в аварию, где он единственный участник;
    * +3% то же самое, но водитель не только пьян, но и под наркотиками;
    * +9% - то же самое, но участников двое, пьян как минимум один из них;
    * +8% - то же самое, но участников двое, кто-то и пьян и употребил наркотики;

## Подготовка датасета для обучения моделей
### Сохранение SQL таблиц  в датасет pandas
Выведем все необходимые поля, а также информацию о времени и месяце происшествия. 

Выше выяснилось, что данные за 2012 год условно полны до мая включительно. В связи с этим колонку с датой происшествия убираем на этапе предобработки датасета в pandas. А колонку со временем происшествия преобразуем в колонку категориального типа, оставив значения "утро", "день", "вечер", "ночь", на этапе выгрузки данных их SQL базы.

In [9]:
query = '''
SELECT *,

    CASE
        WHEN extract(hour from collision_time) IN (00.00, 01.00, 02.00, 03.00, 04.00, 05.00)
        THEN 'night'
        WHEN extract(hour from collision_time) IN (06.00, 07.00, 08.00, 09.00, 10.00, 11.00)
        THEN 'morning'
        WHEN extract(hour from collision_time) IN (12.00, 13.00, 14.00, 15.00, 16.00, 17.00)
        THEN 'day'
        WHEN extract(hour from collision_time) IN (18.00, 19.00, 20.00, 21.00, 22.00, 23.00)
        THEN 'evening'
        ELSE 'unknown'
    END AS time
    
FROM collisions
JOIN parties ON collisions.case_id = parties.case_id
LEFT JOIN vehicles ON vehicles.case_id = parties.case_id AND vehicles.party_number = parties.party_number

WHERE parties.party_type = 'car'
AND collisions.collision_damage != 'scratch'
AND collisions.collision_date > '2012-01-01'
AND collisions.collision_date < '2013-01-01'
ORDER BY collisions.collision_date
'''

df = pd.read_sql_query(query, con=engine) 
df

,case_id,county_city_location,county_location,distance,direction,intersection,weather_1,location_type,collision_damage,party_count,...,party_sobriety,party_drug_physical,cellphone_in_use,id,case_id,party_number,vehicle_type,vehicle_transmission,vehicle_age,time
0,5473173,4700,siskiyou,"4,752.0000",south,0.0000,clear,highway,small damage,2,...,had not been drinking,None,0.0000,NaN,None,NaN,None,None,NaN,morning
1,5479970,1973,los angeles,108.0000,west,0.0000,clear,None,small damage,2,...,had not been drinking,None,NaN,NaN,None,NaN,None,None,NaN,morning
2,5480523,5600,ventura,"8,976.0000",north,0.0000,clear,highway,severe damage,2,...,had not been drinking,None,0.0000,"1,352,500.0000",5480523,2.0000,coupe,manual,7.0000,day
3,5494865,1942,los angeles,30.0000,east,0.0000,clear,None,small damage,2,...,had not been drinking,None,0.0000,NaN,None,NaN,None,None,NaN,evening
4,5455519,3010,orange,917.0000,south,0.0000,fog,highway,small damage,2,...,had not been drinking,None,0.0000,NaN,None,NaN,None,None,NaN,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195893,5639838,1931,los angeles,0.0000,None,1.0000,clear,None,fatal,2,...,"had been drinking, under influence",None,0.0000,NaN,None,NaN,None,None,NaN,evening
195894,5639421,3404,sacramento,28.0000,east,0.0000,clear,ramp,fatal,2,...,impairment unknown,G,0.0000,"1,504,187.0000",5639421,1.0000,coupe,manual,14.0000,day
195895,5520402,0103,alameda,200.0000,east,0.0000,clear,highway,small damage,2,...,had not been drinking,None,0.0000,NaN,None,NaN,None,None,NaN,evening
195896,5639994,3496,sacramento,211.0000,north,0.0000,clear,None,fatal,2,...,had not been drinking,under drug influence,0.0000,NaN,None,NaN,None,None,NaN,day


## Предобработка данных
### Удаление неинформативных колонок

Сразу удалим лишние колонки `case_id`, `id`. Колонка `party_number` была нужна только для связи двух таблиц, а колонки `county_city_location`, `county_location` не пригодятся, т.к. данные из одного региона, а задача не выявить зависимость риска от локации внутри региона, а экстраполировать данные региона на любой другой регион. Направление движения `direction`, вероятней всего, имеет значение только для региона, удалим его.

Колонки 'collision_date' и 'collision_time' также удалены.

In [10]:
# сохраним резервную копию датасета
df_original = df 

На этом этапе дубликатов нет. Они появятся впоследствии, но удалять их не будем.

In [11]:
# df = df_original

In [12]:
print(df.columns)
df = df.drop(['case_id', 'id', 'party_number', 'party_type', 
              'county_city_location', 'county_location', 'direction', 'collision_date', 'collision_time'], axis=1)
df.columns

Index(['case_id', 'county_city_location', 'county_location', 'distance',
       'direction', 'intersection', 'weather_1', 'location_type',
       'collision_damage', 'party_count', 'primary_collision_factor',
       'pcf_violation_category', 'type_of_collision',
       'motor_vehicle_involved_with', 'road_surface', 'road_condition_1',
       'lighting', 'control_device', 'collision_date', 'collision_time', 'id',
       'case_id', 'party_number', 'party_type', 'at_fault',
       'insurance_premium', 'party_sobriety', 'party_drug_physical',
       'cellphone_in_use', 'id', 'case_id', 'party_number', 'vehicle_type',
       'vehicle_transmission', 'vehicle_age', 'time'],
      dtype='object')


Index(['distance', 'intersection', 'weather_1', 'location_type',
       'collision_damage', 'party_count', 'primary_collision_factor',
       'pcf_violation_category', 'type_of_collision',
       'motor_vehicle_involved_with', 'road_surface', 'road_condition_1',
       'lighting', 'control_device', 'at_fault', 'insurance_premium',
       'party_sobriety', 'party_drug_physical', 'cellphone_in_use',
       'vehicle_type', 'vehicle_transmission', 'vehicle_age', 'time'],
      dtype='object')

### Предварительная оценка датасета
Получился датасет на 195_898 строк и 23 колонки. 
#### Колонки
* 'distance'
* 'intersection' 
* 'weather_1' 
* 'location_type'
* 'collision_damage' 
* 'party_count' 
* 'primary_collision_factor'
* 'pcf_violation_category'
* 'type_of_collision'
* 'motor_vehicle_involved_with' 
* 'road_surface' 
* 'road_condition_1',
* 'lighting' 
* 'control_device' 
* 'party_type' 
* 'at_fault' 
* 'insurance_premium'
* 'party_sobriety'
* 'party_drug_physical'
* 'cellphone_in_use'
* 'vehicle_type'
* 'vehicle_transmission' 
* 'vehicle_age'
* 'time'

В данных есть пропуски, выбросы. После удаления колонок появились дубликаты, однако удалять их не будем т.к. они соответствуют разным событиям.

In [13]:
df.duplicated().sum()

12671

#### Отбор колонок для обучения модели
Выберем колонки для обучения модели:

* 'at_fault' - целевой признак - виновность водителя в ДТП;

Бинарные признаки:
* 'intersection' - произошло ли событие на перекрестке;
* 'cellphone_in_use' - наличие телефона может коррелировать с моделью автомобиля, а может отвлекать водителя;

Численные признаки:
* 'distance' - расстояние от главной дороги;
* 'party_count' - количество сторон в ДТП влияет на риск водителя оказаться виновным (если он единственный участник ДТП, то точно ответственный за ДТП);
* 'insurance_premium' - сумма страховки может быть связана с поведением водителя;
* 'vehicle_age' - возраст автомобиля может влиять на его исправность;

Категориальные признаки:
* 'weather' - погода в момент ДТП;
* 'location_type' - тип местности, где произошло ДТП;
* 'collision_damage' - серьезность происшествия;
* 'primary_collision_factor' - основной фактор аварии;
* 'pcf_violation_category' - категория нарушения, допущенного водителем;
* 'type_of_collision' - тип происшествия (тоже может влиять: кто-то убился в одиночку, под кого-то бросился пешеход, кто-то вылетел с трассы и тд);
* 'motor_vehicle_involved_with' - второй участник ДТП при его наличии;
* 'road_surface' - поверхность дороги (влажная, сухая и тд);
* 'road_condition' - состояние дорожного полотна (ямы, дыры, ремонт, отсутствие);
* 'lighting' - освещение;
* 'control_device' - устройство управления автомобилем, указано наличие и исправность;
* 'party_sobriety' - степень трезвости/опьянения водителя;
* 'party_drug_physical' - степень наркотического опьянения, проблемы со здоровьем водителя;
* 'vehicle_type' - тип транспортного средства;
* 'vehicle_transmission' - тип КПП транспотного средства (конструктивные особенности могут влиять на вероятность аварии)
* 'time' - время суток

В датасете были временные признаки:

Временные признаки:
* 'collision_date'
* 'collision_time'

Информацию из 'collision_time' сохранили в колонку 'time', а исходную колонку удалили. Колонку 'collision_date' удалили, не сохраняя эту информацию в датасете в другом виде: данные по месяцам неполны, обучение по дате может исказить выводы.


#### Переименование колонок
Для удобства укоротим названия колонок:

In [14]:
df = df.rename(columns={'weather_1': 'weather', 'road_condition_1': 'road_condition'})

#### Заполнение пустых значений и объединение колонок

В колонках `cellphone_in_use`, `intersection` заменим отсутствующие значения на нули: если нет информации, значит, скорей всего, нет и телефона в машине, или событие произошло не на перекрестке.

In [15]:
df.loc[:, ['cellphone_in_use', 'intersection']] = df.loc[:, ['cellphone_in_use', 'intersection']].fillna(0.0)

Объединим колонки `location_type` и `intersection`. Информацию сохраним в колонке `location_type`, колонку `intersection` удалим.

In [16]:
# напишем функцию, которая возвращает новое значение
def new_location_type(row):
    location_type = 'unknown'
    
    if row['intersection'] == 1:
        united = [row['location_type'], 'intersection']
        location_type = [el for el in united if el is not None]
        location_type = ', '.join(location_type)
    elif row['location_type'] is not None:
        location_type = row['location_type']
        
    if location_type == 'intersection, intersection':
        location_type = 'intersection'

    return location_type

In [17]:
df['new_location_type'] = df.apply(lambda row: new_location_type(row), axis=1)
df['location_type'] = df['new_location_type']
df['location_type'].value_counts()
df = df.drop(['new_location_type', 'intersection'], axis=1)

In [18]:
print(df.location_type.isna().mean())
df.location_type.value_counts()

0.0


unknown                  74951
highway                  70819
intersection             37830
ramp                      9711
ramp, intersection        1754
highway, intersection      833
Name: location_type, dtype: int64

В колонке `insurance_premium`, `vehicle_age` заменим пустые значения медианным (хотя заполнять 70% данных в колонке `vehicle_age` медианными страшно):

In [19]:
df['insurance_premium'] = df['insurance_premium'].fillna(df['insurance_premium'].median())
df['vehicle_age'] = df['vehicle_age'].fillna(df['vehicle_age'].median())

В колонке `party_drug_physical` отсутствует более 87% значений. Объединим эту колонку с колонкой `party_sobriety` в новую колонку `driver_condition`.

In [20]:
# напишем функцию, которая возвращает новое значение
def define_driver_condition(row):
    united = [row['party_sobriety'], row['party_drug_physical']]
    condition = [el for el in united if el is not None]
    condition = ', '.join(condition)
    if condition is None or condition == '':
        # почему-то не работает
        condition == 'unknown'
    return condition

In [21]:
df['driver_condition'] = df.apply(lambda row: define_driver_condition(row), axis=1)
df = df.drop('party_sobriety', axis=1)
df = df.drop('party_drug_physical', axis=1)

In [22]:
df['driver_condition'].value_counts()

had not been drinking                                            156225
impairment unknown, G                                             20365
had been drinking, under influence                                 9796
                                                                   1982
not applicable, not applicable                                     1667
had been drinking, impairment unknown                              1652
had been drinking, not under influence                             1284
under drug influence                                                850
had not been drinking, sleepy/fatigued                              508
sleepy/fatigued                                                     501
had not been drinking, under drug influence                         329
had been drinking, under influence, under drug influence            279
impairment - physical                                               268
had not been drinking, impairment - physical                    

В других категориальных колонках заменим пустые значения на 'unknown':

In [23]:
categorical_columns = df.select_dtypes(include=['object']).columns
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
(len(categorical_columns) + len(numerical_columns)) == len(df.columns)

True

In [24]:
for column in categorical_columns:
    df[column] = df[column].fillna('unknown')

Нужно отметить, что в колонках, связанных с состоянием и конструктивными особенностями автомобиля до заполнения пустых значений было более 70% отсутствующих значений.

In [25]:
df.isna().mean()

distance                      0.0000
weather                       0.0000
location_type                 0.0000
collision_damage              0.0000
party_count                   0.0000
primary_collision_factor      0.0000
pcf_violation_category        0.0000
type_of_collision             0.0000
motor_vehicle_involved_with   0.0000
road_surface                  0.0000
road_condition                0.0000
lighting                      0.0000
control_device                0.0000
at_fault                      0.0000
insurance_premium             0.0000
cellphone_in_use              0.0000
vehicle_type                  0.0000
vehicle_transmission          0.0000
vehicle_age                   0.0000
time                          0.0000
driver_condition              0.0000
dtype: float64

#### Замена типов колонок

Тип колонок 'party_count', 'insurance_premium', 'vehicle_age' заменим на целочисленный. Тип колонки 'cellphone_in_use' можно заменить на `boolean`, но это сделаем после оценки корреляций, пока переведем ее в целочисленный тип.

In [26]:
df.loc[:, ['party_count', 
           'insurance_premium', 
           'vehicle_age',  
           'cellphone_in_use',
           'at_fault']] = df.loc[:,   ['party_count', 
                                       'insurance_premium', 
                                       'vehicle_age', 
                                       'cellphone_in_use',
                                       'at_fault']].astype('int32')

Сформируем списки колонок по типам данных:

In [27]:
categorical_columns = df.select_dtypes(include=['object']).columns
numerical_columns = df.select_dtypes(include=['float64', 'int64', 'int32']).columns
boolean_columns = df.select_dtypes(include=['bool']).columns

(len(categorical_columns) + 
    len(numerical_columns) + 
    len(boolean_columns)) == len(df.columns)

True

In [28]:
print(categorical_columns)
print(numerical_columns)
print(boolean_columns)

Index(['weather', 'location_type', 'collision_damage',
       'primary_collision_factor', 'pcf_violation_category',
       'type_of_collision', 'motor_vehicle_involved_with', 'road_surface',
       'road_condition', 'lighting', 'control_device', 'vehicle_type',
       'vehicle_transmission', 'time', 'driver_condition'],
      dtype='object')
Index(['distance', 'party_count', 'at_fault', 'insurance_premium',
       'cellphone_in_use', 'vehicle_age'],
      dtype='object')
Index([], dtype='object')


Пока бинарных колонок нет, тип данных изменим позже.

#### Обработка выбросов
Проверим численные колонки на выбросы, обработаем их. Явные выбросы есть только в колонке `vehicle_age`. Заменим их на медианное значение по колонке.

In [29]:
sns.set(rc={'figure.figsize':(10,5)})

def print_hist_for_one_column(column_name, title='График распределения', bins=50):
    ax = df[column_name].hist(bins=bins)
    ax.set_title(title)
    ax.set_xlabel(column_name)
    ax.set_ylabel('Объекты датасета')
    plt.show()
    plt.clf()

def print_boxplot(column_name, title='Ящик с усами'):
    sns.boxplot(x=df[column_name])
    plt.title(title)
    plt.show()
    plt.clf()
    


In [30]:
# До замены:
print(df.vehicle_age.value_counts())
df.loc[df['vehicle_age'] > 20, 'vehicle_age'] = df.vehicle_age.median()

4      148792
3       10925
2        6007
5        5534
6        3922
7        3814
8        3484
0        3076
9        2748
1        2533
10       1938
11       1369
12        877
13        549
14        281
15         37
16          6
17          3
161         2
19          1
Name: vehicle_age, dtype: int64


#### Укрупнение категорий
Проверим значения в категориальных колонках и при необходимости укрупним категории.

In [31]:
def print_categorical_columns_info():
    all_uniq_categorical_count = 0
    for column in categorical_columns:
        print(column)
        print('Uniq values:', len(df[column].unique()))
        all_uniq_categorical_count += len(df[column].unique())
        print(df[column].value_counts())
        print(); print()

    print('Всего уникальных категориальных значений:', all_uniq_categorical_count)
    
print_categorical_columns_info()

weather
Uniq values: 8
clear      156523
cloudy      29469
raining      8110
unknown       649
fog           492
snowing       433
other         175
wind           47
Name: weather, dtype: int64


location_type
Uniq values: 6
unknown                  74951
highway                  70819
intersection             37830
ramp                      9711
ramp, intersection        1754
highway, intersection      833
Name: location_type, dtype: int64


collision_damage
Uniq values: 4
small damage     159383
middle damage     27916
severe damage      4897
fatal              3702
Name: collision_damage, dtype: int64


primary_collision_factor
Uniq values: 5
vehicle code violation    186050
unknown                     5584
other than driver           3356
other improper driving       902
fell asleep                    6
Name: primary_collision_factor, dtype: int64


pcf_violation_category
Uniq values: 22
speeding                             69100
improper turning                     28522
automobi

Объединять значения unknown и other в колонках не будем. Все же они имеют разное происхождение. Укрупним значения в колонках `pcf_violation_category` и `road_condition`.

In [32]:
def remove_small_cats(df, column_name, threshold):
    aggregated_dict = dict()
    for (category, count) in df[column_name].value_counts().to_dict().items():
        if count >= threshold:
            aggregated_dict[category] = category
        else:
            aggregated_dict[category] = 'other'
 
    df['new'] = df[column_name].map(aggregated_dict)
    df[column_name] = df['new']
    df = df.drop('new', axis=1)
    print(df[column_name].value_counts())
    return df
    

In [33]:
df = remove_small_cats(df, 'road_condition', 2000)

normal          188647
other             3903
construction      3348
Name: road_condition, dtype: int64


In [34]:
df = remove_small_cats(df, 'pcf_violation_category', 2000)

speeding                             69100
improper turning                     28522
automobile right of way              20930
unsafe lane change                   18347
dui                                  14034
traffic signals and signs             9115
unknown                               8797
unsafe starting or backing            7644
other                                 7580
following too closely                 4550
wrong side of road                    3923
other than driver (or pedestrian)     3356
Name: pcf_violation_category, dtype: int64


Укрупним значения в колонке `driver_condition`. Судя по https://tims-test.berkeley.edu/help/SWITRS.php, 'G' соответствует 'impairment unknown', произведем эту замену. Изначально в колонке `driver_condition` 22 уникальных значения. Составим словарь для замены и произведем замену. После замены осталось 12 уникальных значений.

In [35]:
print(len(df['driver_condition'].unique()))
df['driver_condition'].value_counts()

22


had not been drinking                                            156225
impairment unknown, G                                             20365
had been drinking, under influence                                 9796
                                                                   1982
not applicable, not applicable                                     1667
had been drinking, impairment unknown                              1652
had been drinking, not under influence                             1284
under drug influence                                                850
had not been drinking, sleepy/fatigued                              508
sleepy/fatigued                                                     501
had not been drinking, under drug influence                         329
had been drinking, under influence, under drug influence            279
impairment - physical                                               268
had not been drinking, impairment - physical                    

In [36]:
driver_conditions_dict = {'had not been drinking': 'had not been drinking',

                     'had been drinking, under influence':  'had been drinking, under influence',
                     'had been drinking, impairment unknown': 'had been drinking, impairment unknown',
                     'had been drinking, not under influence': 'had been drinking, not under influence',
                        
                     'impairment unknown, G': 'unknown', 
                     '': 'unknown',
                     'not applicable, not applicable': 'not applicable',
                          

                     'under drug influence': 'under drug influence',
            
                     'sleepy/fatigued': 'sleepy/fatigued',                
                     'impairment - physical': 'impairment - physical',
                          
                     'had not been drinking, impairment - physical': 'impairment - physical',
                     'had not been drinking, sleepy/fatigued': 'sleepy/fatigued',
                     'had not been drinking, under drug influence':  'under drug influence',
                     
                     'had been drinking, under influence, under drug influence': 'had been drinking, under drug influence',
                     'had been drinking, impairment unknown, under drug influence': 'had been drinking, under drug influence',
                     'had been drinking, not under influence, under drug influence': 'had been drinking, under drug influence',
                          
                     'had been drinking, under influence, sleepy/fatigued': 'had been drinking, sleepy/fatigued',
                     'had been drinking, not under influence, sleepy/fatigued': 'had been drinking, sleepy/fatigued',
                     'had been drinking, impairment unknown, sleepy/fatigued': 'had been drinking, sleepy/fatigued',
                     
                     'had been drinking, not under influence, impairment - physical': 'had been drinking, impairment - physical',
                     'had been drinking, under influence, impairment - physical': 'had been drinking, impairment - physical',
                     'had been drinking, impairment unknown, impairment - physical': 'had been drinking, impairment - physical'}

df['driver_condition'] = df['driver_condition'].map(driver_conditions_dict)
print(len(df['driver_condition'].unique()))
df['driver_condition'].value_counts()

12


had not been drinking                       156225
unknown                                      22347
had been drinking, under influence            9796
not applicable                                1667
had been drinking, impairment unknown         1652
had been drinking, not under influence        1284
under drug influence                          1179
sleepy/fatigued                               1009
impairment - physical                          331
had been drinking, under drug influence        328
had been drinking, sleepy/fatigued              74
had been drinking, impairment - physical         6
Name: driver_condition, dtype: int64

In [37]:
print_categorical_columns_info()

weather
Uniq values: 8
clear      156523
cloudy      29469
raining      8110
unknown       649
fog           492
snowing       433
other         175
wind           47
Name: weather, dtype: int64


location_type
Uniq values: 6
unknown                  74951
highway                  70819
intersection             37830
ramp                      9711
ramp, intersection        1754
highway, intersection      833
Name: location_type, dtype: int64


collision_damage
Uniq values: 4
small damage     159383
middle damage     27916
severe damage      4897
fatal              3702
Name: collision_damage, dtype: int64


primary_collision_factor
Uniq values: 5
vehicle code violation    186050
unknown                     5584
other than driver           3356
other improper driving       902
fell asleep                    6
Name: primary_collision_factor, dtype: int64


pcf_violation_category
Uniq values: 12
speeding                             69100
improper turning                     28522
automobi

По результатам укрупнения категорий, осталось 96 уникальных значений в категориальных колонках.

### Оценка предобработанного датасета
В ходе предобработки данных было проведено переименование колонок, заполнение пустых значений, изменение типов колонок на ниболее подходящие, очистка числовых данных от выбросов (колонка 'vehicle_age') и укрупнение категорий в категориальных колонках.

В итоге предобработки получился датасет на 196_320 строк и 21 колонку. В категориальных колонках осталось 96 уникальных значений, возможно, что-то из этого впоследствии нужно будет укрупнить или удалить. 

Матрица корреляций не показывает никаких значимых линейных корреляций целевого признака с обучающими.

Корреляция Спирмена в основном повторяет результат корреляции Пирсона, указывая на слабую отрицательную корреляцию целевого признака 'at_fault' с 'party_count' (Kspearman = -0.28, Kpearson = -0.26) и еще более слабую корреляцию с крупной суммой страховки ('insurance_premium'): Kspearman = -0.14, Kpearson = -0.12. 

Кроме того есть положительная корреляция между суммой страховки и возрастом ТС (Kspearman = 0.32, Kpearson = 0.34).

In [38]:
# функция печати матрицы диаграмм рассеяния
def scatter_matrix(dataframe):
    pd.plotting.scatter_matrix(
        dataframe, 
        figsize=(12, 12)
    )

In [39]:
# функция печати тепловой карты коэффициентов корреляции
def correlation_heatmap(dataframe):
    plt.figure(figsize=(18, 7))

    sns.heatmap(dataframe, annot=True, fmt=".5f", linewidths=.1, cmap='RdYlGn')
    plt.title('Матрица коэффициентов корреляции', fontsize=15)
    plt.ylabel('Признак', fontsize=15)
    plt.xlabel('Признак', fontsize=15)

### Замена типа колонок на boolean в колонке `cellphone_in_use`
А теперь заменим тип колонок на boolean в колонке `cellphone_in_use`.

In [40]:
df.loc[:, ['cellphone_in_use']] = df.loc[:, ['cellphone_in_use']].astype('bool')

In [41]:
categorical_columns = df.select_dtypes(include=['object']).columns
numerical_columns = df.select_dtypes(include=['float64', 'int64', 'int32']).columns
boolean_columns = df.select_dtypes(include=['bool']).columns

(len(categorical_columns) + 
    len(numerical_columns) + 
    len(boolean_columns)) == len(df.columns)

True

In [42]:
print(categorical_columns)
print(numerical_columns)
print(boolean_columns)

Index(['weather', 'location_type', 'collision_damage',
       'primary_collision_factor', 'pcf_violation_category',
       'type_of_collision', 'motor_vehicle_involved_with', 'road_surface',
       'road_condition', 'lighting', 'control_device', 'vehicle_type',
       'vehicle_transmission', 'time', 'driver_condition'],
      dtype='object')
Index(['distance', 'party_count', 'at_fault', 'insurance_premium',
       'vehicle_age'],
      dtype='object')
Index(['cellphone_in_use'], dtype='object')


## Подготовка датасетов для обучения моделей
Подготовим датасеты для обучения моделей. Существенного дисбаланса классов в исходном датасете нет.

In [43]:
df['at_fault'].value_counts()

1    102453
0     93445
Name: at_fault, dtype: int64

### Разделение на тренировочный, тестовый, валидационный датасеты
Разделим датасет на тренировочный, валидационный и тестовый. Выделим обучающие и целевой признаки.

In [44]:
def make_train_test_valid_dfs(df, target):
    features = df.drop([target], axis=1)
    target = df[target]

    features_train, features_test_and_valid, target_train, target_test_and_valid = train_test_split(
        features, target, test_size=0.4, random_state=666)

    features_valid, features_test, target_valid, target_test = train_test_split(
        features_test_and_valid, target_test_and_valid, test_size=0.5, random_state=666)

    print('Количество объектов в features_train:', len(features_train))
    print('Количество объектов в target_train:', len(target_train))

    print('Количество объектов в features_valid:', len(features_valid))
    print('Количество объектов в target_valid:', len(target_valid))

    print('Количество объектов в features_test:', len(features_test))
    print('Количество объектов в target_test:', len(target_test))
    
    return features_train, target_train, features_valid, target_valid, features_test, target_test

In [45]:
target = 'at_fault'
features_train, target_train, features_valid, target_valid, features_test, target_test = make_train_test_valid_dfs(df, target)

Количество объектов в features_train: 117538
Количество объектов в target_train: 117538
Количество объектов в features_valid: 39180
Количество объектов в target_valid: 39180
Количество объектов в features_test: 39180
Количество объектов в target_test: 39180


### Масштабирование численных признаков
Масштабируем численные признаки.

In [46]:
numerical_columns = features_train.select_dtypes(include=['float64', 'int64', 'int32']).columns
numerical_columns

Index(['distance', 'party_count', 'insurance_premium', 'vehicle_age'], dtype='object')

In [47]:
scaler = StandardScaler()
scaler.fit(features_train[numerical_columns])

for dataset in [features_train, features_valid, features_test]:
    dataset[numerical_columns] = scaler.transform(dataset[numerical_columns])

features_train.head()

,distance,weather,location_type,collision_damage,party_count,primary_collision_factor,pcf_violation_category,type_of_collision,motor_vehicle_involved_with,road_surface,road_condition,lighting,control_device,insurance_premium,cellphone_in_use,vehicle_type,vehicle_transmission,vehicle_age,time,driver_condition
159669,-0.0702,clear,highway,small damage,3.9828,vehicle code violation,dui,rear end,other motor vehicle,dry,normal,daylight,none,0.9332,False,unknown,unknown,-0.1424,morning,had not been drinking
161449,-0.0707,clear,intersection,small damage,-0.1487,vehicle code violation,automobile right of way,rear end,other motor vehicle,dry,normal,daylight,functioning,-1.3395,False,unknown,unknown,-0.1424,day,had not been drinking
193846,0.8221,clear,highway,fatal,-0.1487,vehicle code violation,improper turning,broadside,motor vehicle on other roadway,dry,normal,daylight,none,-0.2356,False,coupe,auto,0.4520,morning,had not been drinking
192471,-0.0596,clear,unknown,small damage,1.2285,vehicle code violation,speeding,rear end,other motor vehicle,dry,normal,daylight,none,-0.1707,False,unknown,unknown,-0.1424,evening,unknown
55423,-0.0628,raining,unknown,small damage,-0.1487,vehicle code violation,improper turning,sideswipe,other motor vehicle,wet,normal,daylight,none,0.4787,False,unknown,unknown,-0.1424,morning,had not been drinking


### Изменение типа категориальных колонок для LightGBM
CatBoost способен работать с категориальными данными напрямую благодаря встроенному преобразованию данных. А для LightGBM нужно заменить тип категориальных колонок с 'object' на 'category'. 


In [48]:
features_train_lgb = features_train
features_valid_lgb = features_valid
features_test_lgb = features_test

for column in categorical_columns:
    features_train_lgb[column] = features_train_lgb[column].astype('category')
    features_valid_lgb[column] = features_valid_lgb[column].astype('category')
    features_test_lgb[column] = features_test_lgb[column].astype('category')

In [49]:
features_train_lgb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117538 entries, 159669 to 75650
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   distance                     117538 non-null  float64 
 1   weather                      117538 non-null  category
 2   location_type                117538 non-null  category
 3   collision_damage             117538 non-null  category
 4   party_count                  117538 non-null  float64 
 5   primary_collision_factor     117538 non-null  category
 6   pcf_violation_category       117538 non-null  category
 7   type_of_collision            117538 non-null  category
 8   motor_vehicle_involved_with  117538 non-null  category
 9   road_surface                 117538 non-null  category
 10  road_condition               117538 non-null  category
 11  lighting                     117538 non-null  category
 12  control_device               117538 non-

### Подготовка датасетов с кодированием категориальных признаков для логистической регрессии
Для логистической регрессии нужно будет кодировать категориальные признаки. Сделаем это методом OHE, как наиболее универсальным.

In [50]:
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False, dtype='int32')
ohe.fit(features_train[categorical_columns])

def get_ohe(dataset):
    temp_dataset = pd.DataFrame(
        data=ohe.transform(dataset[categorical_columns]), 
        columns=ohe.get_feature_names_out())
 
    dataset.drop(columns=categorical_columns, axis=1, inplace=True)
    
    dataset = pd.concat([dataset.reset_index(drop=True), temp_dataset], axis=1)
    return dataset

In [51]:
features_train_ohe = features_train.copy()
features_valid_ohe = features_valid.copy()
features_test_ohe = features_test.copy()

features_train_ohe = get_ohe(features_train_ohe)
features_valid_ohe = get_ohe(features_valid_ohe)
features_test_ohe = get_ohe(features_test_ohe)

In [52]:
features_train_ohe.head()

,distance,party_count,insurance_premium,cellphone_in_use,vehicle_age,weather_cloudy,weather_fog,weather_other,weather_raining,weather_snowing,...,"driver_condition_had been drinking, not under influence","driver_condition_had been drinking, sleepy/fatigued","driver_condition_had been drinking, under drug influence","driver_condition_had been drinking, under influence",driver_condition_had not been drinking,driver_condition_impairment - physical,driver_condition_not applicable,driver_condition_sleepy/fatigued,driver_condition_under drug influence,driver_condition_unknown
0,-0.0702,3.9828,0.9332,False,-0.1424,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,-0.0707,-0.1487,-1.3395,False,-0.1424,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0.8221,-0.1487,-0.2356,False,0.4520,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,-0.0596,1.2285,-0.1707,False,-0.1424,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,-0.0628,-0.1487,0.4787,False,-0.1424,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


Для обучения регрессии готов датасет на 86 колонок.

## Обучение моделей
Обучим модели на подготовленных датасетах. 

В качестве главной метрики будем оценивать F1 метрику. F1 - это среднее гармоническое полноты и точности, то есть она учитывает одновременно долю угаданных объектов положительного класса и долю истинно положительных объектов, среди определенных как положительные. F1 хороша, когда нас интересует в первую очередь положительный класс.

Дополнительно будем отслеживать AUC ROC score - площадь под кривой ошибок. Если важна точность определения моделью обоих классов, положительного и отрицательного, то AUC ROC лучше, т.к. ее сложней обмануть, чем F1. В частности она показывает, с какой вероятностью объект положительного класса будет оценен моделью выше, чем объект отрицательного класса.

### Dummy Classifier
Для проверки на адекватность предскажем целевой признак с помощью DummyClassifier и выведем его метрики. AUC ROC для него равен 0.5.

In [53]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train_ohe, target_train)
dummy_predicted_valid = dummy_clf.predict(features_valid_ohe)
dummy_f1 = f1_score(target_valid, dummy_predicted_valid)
print('f1 =', dummy_f1)

dummy_auc_roc = roc_auc_score(target_valid, dummy_predicted_valid)
print('auc_roc = ', dummy_auc_roc)

f1 = 0.6876350298949907
auc_roc =  0.5


### Логистическая регрессия
Обучим логистическую регрессию и выведем ее метрики.

In [54]:
regression_model = LogisticRegression(random_state=666, solver='liblinear')
regression_model.fit(features_train_ohe, target_train)
predicted_valid_ohe = regression_model.predict(features_valid_ohe)

regression_f1 = f1_score(target_valid, predicted_valid_ohe)
print('f1 =', regression_f1)

regression_auc_roc = roc_auc_score(target_valid, predicted_valid_ohe)
print('auc_roc = ', regression_auc_roc)

f1 = 0.6923410718160121
auc_roc =  0.7149306974955095


In [55]:
f1_scorer_2 = make_scorer(f1_score, greater_is_better=True)

In [56]:
%%time
estimator = LogisticRegression(random_state=666)

params = {  'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
            'max_iter': [50, 100, 500]
         }

grid_2 = GridSearchCV(estimator, 
                    params,
                    scoring=f1_scorer_2,
                    refit='F1',
                    cv=3, 
                    verbose=1,
                    n_jobs=-1) 

grid_2.fit(features_train_ohe, target_train) 

Fitting 3 folds for each of 18 candidates, totalling 54 fits


KeyboardInterrupt: 

In [57]:
print(grid_2.best_params_)
print(grid_2.best_score_)
print(f1_score(target_valid, grid_2.predict(features_valid_ohe)))
print(roc_auc_score(target_valid, grid_2.predict(features_valid_ohe)))

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

Инициализируем модель с параметрами, подобранными GridSearchCV и посчитаем ее метрики. Результат практически идентичный тому, что был.

In [ ]:
regression_model_1 = LogisticRegression(random_state=666, solver='saga', max_iter=500)
regression_model_1.fit(features_train_ohe, target_train)
predicted_valid_ohe = regression_model_1.predict(features_valid_ohe)

regression_1_f1 = f1_score(target_valid, predicted_valid_ohe)
print('f1 =', regression_1_f1)

regression_1_auc_roc = roc_auc_score(target_valid, predicted_valid_ohe)
print('auc_roc = ', regression_1_auc_roc)

### CatBoostClassifier
#### CatBoostClassifier с дефолтными параметрами

Инициализируем модель с дефолтными параметрами и рассчитаем ее метрики.

In [ ]:
%%time
# обучаем на датасете с не кодированными методом OHE категориями
catboost_model = CatBoostClassifier(verbose=0, random_seed=69)
catboost_model.fit(features_train, target_train, cat_features=list(categorical_columns))

In [ ]:
print(catboost_model.get_all_params()['iterations'])
print(catboost_model.get_all_params()['depth'])
print(catboost_model.get_all_params()['max_leaves'])
print(catboost_model.get_all_params()['learning_rate'])

Выведем предсказания, сделанные с помощью стандартного метода model.predict(). Результат получился лучше, чем у регрессии.

In [ ]:
catboost_predicted_valid = catboost_model.predict(features_valid).astype('bool')

catboost_f1 = f1_score(target_valid, catboost_predicted_valid)
print('f1 =', catboost_f1)

catboost_auc_roc = roc_auc_score(target_valid, catboost_predicted_valid)
print('auc_roc = ', catboost_auc_roc)

Выведем предсказания методом model.predict_proba() и рассчитаем метрики. Лучший порог положительного класса - дефолтный 0.4.


In [ ]:
def predict_with_threshold(model, threshold, features):
    probabilities = model.predict_proba(features)
    probabilities_ones = probabilities[:, 1]
    predictions = [(x > threshold) for x in probabilities_ones]
    return predictions

In [ ]:
def choose_threshold(model, features, target, main_metric_name='F1', verbose=False):
    main_metric = 0
    
    for threshold in range(2, 10, 1):
        predictions = predict_with_threshold(model, (threshold*0.1), features)
        this_auc_roc = roc_auc_score(target, predictions)
        this_f1 = f1_score(target, predictions)
        
        if verbose:
            print('Treshold:', threshold*0.1)
            print('F1:', this_f1)
            print('AUC ROC:', this_auc_roc)
            print(); print()

        if main_metric_name == 'AUC_ROC':
            if this_auc_roc > main_metric:
                main_metric = this_auc_roc
                best_threshold = threshold
                secondary_metric_name = 'F1'
                secondary_metric = this_f1
        elif main_metric_name == 'F1':
            if this_f1 > main_metric:
                main_metric = this_f1
                best_threshold = threshold
                secondary_metric_name = 'AUC_ROC'
                secondary_metric = this_auc_roc
        else:
            print('Two main metric names available: "AUC_ROC" or "F1", try again')
    
    print('Best_result:')
    print(main_metric_name, ': ', main_metric)
    print(secondary_metric_name, ': ', secondary_metric)
    print('Best threshold:', best_threshold*0.1)
   
    return(main_metric_name, main_metric, secondary_metric_name, secondary_metric, best_threshold*0.1)


In [ ]:
#catboost_predicted_valid = predict_with_threshold(catboost_model, 0.5, features_train)
(catboost_1_main_metric_name,
catboost_1_main_metric,
catboost_1_secondary_metric_name,
catboost_1_secondary_metric,
catboost_1_best_threshold) = choose_threshold(catboost_model, features_valid, target_valid, main_metric_name='F1')

#### Подбор параметров через RandomizedSearchCV
Попробуем подобрать параметры через RandomizedSearchCV. В результате подбора лучшими из рассмотренных оказались параметры: {'learning_rate': 0.05, 'iterations': 100, 'depth': 4}

Результат этой модели несколько хуже, чем модели catboost_model_2.

In [ ]:
# передадим f1_scorer_2 без needs_thresholds=True
#%%time
estimator = CatBoostClassifier(random_state=666,
                              verbose=0,
                              )
                              
params = {'iterations': [100, 200, 1_000, 2_000],
          #'l2_leaf_reg': [3, 5, 9],
          'depth': [4, 10, 20],
          'learning_rate': [0.05, 0.1, 0.5]}
                              
grid = RandomizedSearchCV(estimator, 
                    params,
                    scoring = f1_scorer_2,
                    refit='F1',
                    cv=2, 
                    verbose=3, 
                    n_jobs=-1) 

grid.fit(features_train_ohe, target_train, eval_set=(features_valid_ohe, target_valid)) 

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
print(f1_score(target_valid, grid.predict(features_valid_ohe)))
print(roc_auc_score(target_valid, grid.predict(features_valid_ohe)))

Инициализируем и обучим модель с параметрами, подобранными RandomizedSearchCV (iterations возьмем с запасом). 

RandomizedSearchCV предложил следующие параметры: {'learning_rate': 0.05, 'iterations': 1000, 'depth': 10}

f1 = 0.7140837079942497

Обучать будем на датасете с категориальными признаками, кодированными методом OHE. Параметр iterations возьмем с запасом.

In [ ]:
%%time
catboost_model_4 = CatBoostClassifier(loss_function='Logloss',
                                      eval_metric='F1',
                                      custom_loss=['AUC', 'F1'],
                                      random_seed=666,  
                                      iterations=1500,
                                      learning_rate=0.05,
                                      max_depth=10,
                                      verbose=0)

catboost_model_4.fit(features_train_ohe, target_train, 
                     #cat_features=list(categorical_columns),
                     eval_set=(features_valid_ohe, target_valid),
                     verbose=False,
                     plot=True)

Видим, что переобучение началось примерно на 400й итерации. Метрики, рассчитанные по предсказаниям обученной модели, ниже:

In [ ]:
(catboost_4_main_metric_name,
catboost_4_main_metric,
catboost_4_secondary_metric_name,
catboost_4_secondary_metric,
catboost_4_best_threshold) = choose_threshold(catboost_model_4, features_valid_ohe, target_valid)

In [ ]:
# посчитали другим методом, без учета трешхолда
catboost_predicted_valid = catboost_model_4.predict(features_valid_ohe)

catboost_auc_roc = roc_auc_score(target_valid, catboost_predicted_valid)
print('auc_roc = ', catboost_auc_roc)

catboost_f1 = f1_score(target_valid, catboost_predicted_valid)
print('f1 =', catboost_f1)

In [ ]:
catboost_model_4.get_all_params()

#### CatBoost модель без eval_metric='F1'
Инициализируем модель с этими же параметрами, не передавая ей `eval_metric='F1'`. Можно сделать вывод, что модель отработала чуть хуже.

In [ ]:
%%time
catboost_model_5 = CatBoostClassifier(loss_function='Logloss',
                                      #eval_metric='F1',
                                      custom_loss=['AUC', 'F1'],
                                      random_seed=666,  
                                      iterations=1500,
                                      learning_rate=0.05,
                                      max_depth=10,
                                      verbose=0)

catboost_model_5.fit(features_train_ohe, target_train, 
                     #cat_features=list(categorical_columns),
                     #eval_set=(features_valid_ohe, target_valid),
                     verbose=False,
                     plot=True)

In [ ]:
(catboost_5_main_metric_name,
catboost_5_main_metric,
catboost_5_secondary_metric_name,
catboost_5_secondary_metric,
catboost_5_best_threshold) = choose_threshold(catboost_model_5, features_valid_ohe, target_valid)

In [ ]:
(catboost_5_main_metric_name,
catboost_5_main_metric,
catboost_5_secondary_metric_name,
catboost_5_secondary_metric,
catboost_5_best_threshold) = choose_threshold(catboost_model_5, features_test_ohe, target_test)

### LightGBM
#### LightGBM с дефолтными параметрами
Инициализируем модель LightGBM, обучим ее и оценим результат.

In [ ]:
lgb_model = lgb.LGBMClassifier(random_state=666)
lgb_model.fit(features_train_lgb, target_train, categorical_feature=list(categorical_columns))

Выведем предсказания стандартным методом model.predict() и оценим метрику.

In [ ]:
predictions_with_predict_lgb_valid = lgb_model.predict(features_valid_lgb)
lgb_f1 = f1_score(target_valid, predictions_with_predict_lgb_valid)
print('f1 =', lgb_f1)

lgb_auc_roc = roc_auc_score(target_valid, predictions_with_predict_lgb_valid)
print('auc_roc = ', lgb_auc_roc)

Выведем предсказания методом model.predict_proba() и рассчитаем метрики. Результат получился чуть выше.

In [ ]:
lgb_model.predict_proba(features_valid_lgb)

In [ ]:
(lgb_main_metric_name,
lgb_main_metric,
lgb_secondary_metric_name,
lgb_secondary_metric,
lgb_best_threshold) = choose_threshold(lgb_model, features_valid_lgb, target_valid)

Выведем параметры модели.

In [ ]:
lgb_model.get_params()

#### Подбор параметров LightGBM через RandomizedSearchCV
Попробуем подобрать более оптимальные параметры через RandomizedSearchCV.

In [ ]:
# передадим roc_auc_scorer_2 без needs_thresholds=True
# %%time
estimator = lgb.LGBMClassifier(random_state=666,
                               verbose=0,
                               eval_metric='F1',
                               categorical_feature=list(categorical_columns)
                               )
                              
params = {#'iterations': [100, 200],
          #'l2_leaf_reg': [3, 5, 9],
          'n_estimators': [10, 20, 100, 200],
          'min_data_in_leaf': [50, 100, 200],
          'max_depth': [4, 10, 20],
          'num_iterations': [50, 100, 200, 500, 1000, 1500, 2000, 2500],
          #'learning_rate': [0.05, 0.1, 0.5]
          }
                              
grid = RandomizedSearchCV(estimator, 
                          params,
                          scoring=f1_scorer_2,
                          refit='F1',
                          cv=2, 
                          verbose=0, 
                          n_jobs=12,
                          #learning_rate=increase_it?
                         ) 

grid.fit(features_train_lgb, target_train, eval_metric='f1', eval_set=(features_valid_lgb, target_valid)) 

In [ ]:
print(grid.best_score_)
print(grid.best_params_)
print(f1_score(target_valid, grid.predict(features_valid_lgb)))
print(roc_auc_score(target_valid, grid.predict(features_valid_lgb)))

Инициализируем модель с параметрами, предложенными RandomizedSearchCV, обучим ее и оценим метрику. Параметр num_iterations возьмем с запасом.

In [ ]:
lgb_model_2 = lgb.LGBMClassifier(random_state=666,
                              num_iterations=500,
                              n_estimators=20, 
                              min_data_in_leaf=50,
                              max_depth=4,
                              verbose=0)
lgb_model_2.fit(features_train_lgb, target_train, categorical_feature=list(categorical_columns))

In [ ]:
(lgb_2_main_metric_name,
lgb_2_main_metric,
lgb_2_secondary_metric_name,
lgb_2_secondary_metric,
lgb_2_best_threshold) = choose_threshold(lgb_model_2, features_valid, target_valid)

In [ ]:
lgb_model_2.get_params()

## Выбор и оценка моделей
### Сводная таблица результатов
Сведем метрики моделей на валидационном датасете в одну таблицу. Лучшие результаты показали модели catboost_model_4 и lgb_model_2, параметры для которых подобраны на RandomGridSearch.

In [ ]:
data = {'models': ['regression_model',
                   'regression_model_1',
                   'catboost_model', 
                   'catboost_model_4',
                   'catboost_model_5',
                   'lgb_model',
                   'lgb_model_2'],
        'F1':     [regression_auc_roc, 
                    regression_1_auc_roc,
                    catboost_1_main_metric,
                    catboost_4_main_metric,
                    catboost_5_main_metric,
                    lgb_main_metric,
                    lgb_2_main_metric],
        'AUC ROC':  [regression_f1,
                    regression_1_f1,
                    catboost_1_secondary_metric,
                    catboost_4_secondary_metric,
                    catboost_5_secondary_metric,
                    lgb_secondary_metric,
                    lgb_2_secondary_metric]}
results = pd.DataFrame(data=data)
results

### Проверка ВСЕХ моделей на тесте
Проверим, какие результаты модели дадут на тестовых данных. 

In [ ]:
print('regression_model:')
print(choose_threshold(regression_model, features_test_ohe, target_test, main_metric_name='F1'))
print(); print()

print('regression_model_1:')
print(choose_threshold(regression_model_1, features_test_ohe, target_test, main_metric_name='F1'))
print(); print()

print('catboost_model:')
print(choose_threshold(catboost_model, features_test, target_test, main_metric_name='F1'))
print(); print()

print('catboost_model_4:')
print(choose_threshold(catboost_model_4, features_test_ohe, target_test, main_metric_name='F1'))
print(); print()

print('catboost_model_5:')
print(choose_threshold(catboost_model_5, features_test_ohe, target_test, main_metric_name='F1'))
print(); print()

print('lgb_model:')
print(choose_threshold(lgb_model, features_test_lgb, target_test, main_metric_name='F1'))
print(); print()

print('lgb_model_2:')
print(choose_threshold(lgb_model_2, features_test_lgb, target_test, main_metric_name='F1'))
print(); print()

### Выбор модели
Лучшая модель выбирается исходя из метрик, полученных на валидационном датасете.

Лучшая из получившихся моделей - catboost_model_4, обученная с настройками, подобранными через RandomizedSearchCV. 

Параметры, заданные модели:
* num_iterations=1500,
* learning_rate=0.05
* max_depth=10
* eval_metric='F1'

Все параметры модели:

In [ ]:
catboost_model_4.get_params()

In [ ]:
best_model = catboost_model_4

Метрики модели на валидационных данных:
* F1 :  0.7431756663761068
* AUC_ROC :  0.7105323707217235
* Best threshold: 0.4

При запуске модели на тестовых данных метрика получилась примерно такая же:

* F1 :  0.7388942595405791
* AUC_ROC :  0.7071206946239839
* Best threshold: 0.4

Другие модели на тесте дают похожий результат. 

In [ ]:
choose_threshold(best_model, features_test_ohe, target_test, verbose=True)

In [ ]:
test_preds = best_model.predict(features_test_ohe)
print('F1:', f1_score(test_preds, target_test))
print('AUC ROC:', roc_auc_score(test_preds, target_test))

In [ ]:
#  проверим, что это не ошибка типов данных
def predict_with_threshold_int(model, threshold, features):
    probabilities = model.predict_proba(features)
    probabilities_ones = probabilities[:, 1]
    predictions = [int(x > threshold) for x in probabilities_ones]
    return predictions


test_preds_int = predict_with_threshold_int(best_model, 0.5, features_test_ohe)
target_test_int = target_test.astype('int')

print('F1:', f1_score(test_preds_int, target_test_int))
print('AUC ROC:', roc_auc_score(test_preds_int, target_test_int))

Классы сбалансированы, вряд ли проблема была в этом:

In [ ]:
print(pd.Series(target_train).value_counts())
print(pd.Series(target_valid).value_counts())
print(pd.Series(target_test).value_counts())

### Матрица ошибок, полнота, точность
Выведем матрицу ошибок по результатам предсказаний модели. 

На валидационной выборке: 

In [ ]:
predictions_valid = predict_with_threshold(best_model, 0.4, features_valid_ohe)
print(confusion_matrix(target_valid, predictions_valid))

Как видно из матрицы ошибок, на тестовой выборке модель неверно отнесла объект к отрицательному классу 7_376 раз, а неверно отнесла к положительному - 3_229 раз. 

In [ ]:
predictions_test = predict_with_threshold(best_model, 0.5, features_test_ohe)
print(confusion_matrix(target_test, predictions_test))

Выведем полноту и точность на валидационной выборке:

In [ ]:
print('Полнота:', recall_score(target_valid, predictions_valid))
print('Точность:', precision_score(target_valid, predictions_valid))

Выведем полноту и точность на тестовой выборке. Модель стала хуже определять объекты положительного класса.

In [ ]:
print('Полнота:', recall_score(target_test, predictions_test))
print('Точность:', precision_score(target_test, predictions_test))

### Анализ важности основных факторов
Проанализируем, как модель определила важность основных факторов.

In [ ]:
# For lgb_model:
feature_importances = pd.DataFrame(data=sorted(zip(lgb_model.feature_importances_, features_train_lgb.columns)), 
                                   columns=['Value','Feature'])
print(feature_importances)

plt.figure(figsize=(25, 15))
sns.barplot(x="Value", y="Feature", data=feature_importances.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

Таким образом, модель LGB определила как наиболее важные следующие факторы: 'insurance_premium', 'pcf_violation_category', 'distance' (расстояние от главное дороги). Менее важными показались факторы: 'party_count', 'motor_vehicle_involved_with', 'driver_condition'.

Неважными для модели оказались конструктивные особенности автомобиля и его состояние, погода, освещенность и состояние дороги, время суток, время года, а также тип местности, где произошло ДТП. Неважной оказалась также колонка 'party_drug_physical', в которой отражено состояние водителя, не связанное с АО.

Для сравнения выведем фичи, которые показались самыми важными модели CatBoost.

In [ ]:
# For catboost_model:
feature_importances = dict(zip(list(features_train_ohe.columns), list(catboost_model_4.get_feature_importance())))
feature_importances

In [ ]:
feature_importance = catboost_model_4.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 20))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(features_train_ohe.columns)[sorted_idx])
plt.title('Feature Importance')

Таким образом, модель CatBoost определила как наиболее важные следующие факторы: `primary_collision_factor`, `party_count`, `driver_condition`, `pcf_violation_category`. Важными также показались `motor_vehicle_incolved_with` и `insurance_premium`. Все это факторы, описывающие поведение водителя, а также взаимодействие водителя с другими участниками ДТП (колонки `party_count`, `motor_vehicle_involved_with`). 

Неважными для модели оказались конструктивные особенности автомобиля и его состояние, погода, освещенность и состояние дороги, время суток и сезон год, а также тип местности, где произошло ДТП.

### Дополнительное исследование факторов `party_count`, `insurance_premium`, `distance` 
Корреляция `distance` (расстояние от главной дороги) с целевым признаком почти отсутствует, однако этот фактор был выделен моделью LGB как важный.

Исследуем влияние фактора `party_count` на целевой признак дополнительно. 

Корреляция 'party_count' с целевым признаком была самой сильной в исходном датасете, и действительно, этот признак был выбран моделью CatBoost как важный. Корреляция обратная: чем меньше сторон в ДТП, тем выше риск оказаться виновником. При этом нужно ли убрать эту колонку из датасета, неочевидно.

Можно объяснить обратную корреляцию риска стать виновником ДТП с суммой страховки так: более ответственные водители могут быть склонны страховать автомобиль на более крупные суммы. Эта корреляция, скорей всего, не сбивает модель с толку.

К сожалению, для каршеринга информация о наличии у клиента дорогой страховки на его личный автомобиль может быть недоступна. Хотя можно запрашивать эту информацию на этапе регистрации в системе.

In [ ]:
display(df.loc[:, ['at_fault', 'party_count', 'insurance_premium', 'distance']].corr(method='spearman'))
correlation_heatmap(df.loc[:, ['at_fault', 'party_count', 'insurance_premium', 'distance']].corr(method='spearman'))

In [ ]:
median_insurance_of_faulty_drivers = df.query('at_fault == 1')['insurance_premium'].median()
median_insurance_of_not_faulty_drivers = df.query('at_fault == 0')['insurance_premium'].median()
mean_insurance_of_faulty_drivers = df.query('at_fault == 1')['insurance_premium'].mean()
mean_insurance_of_not_faulty_drivers = df.query('at_fault == 0')['insurance_premium'].mean()

In [ ]:
data = {'sum_of_insurance':   [mean_insurance_of_faulty_drivers,
                               mean_insurance_of_not_faulty_drivers,
                               median_insurance_of_faulty_drivers,
                               median_insurance_of_not_faulty_drivers],
        'at_fault':            ['mean_for_faulty',
                               'mean_for_not_faulty',
                               'median_for_faulty',
                               'median_for_not_faulty']
       }

temp_df = pd.DataFrame(data=data)
temp_df

In [ ]:
print_barplot(temp_df, column_x='at_fault', column_y='sum_of_insurance')

Как видно из графика, сумма страховки для водителей, ставших виновниками ДТП, совсем чуть-чуть ниже, чем для остальных водителей. Причем медианные показатели отличаются еще меньше, чем средние. Возможно, картина отчасти искажена тем, что отсутствующие данные - порядка 10% - были заполнены медианными.

Тем не менее, водителей, с более высокой суммой страховки собственного автомобиля можно считать более надежными.

## Вывод по работе
### Постановка задачи
На основании датасета о ДТП в отдельном регионе нужно разработать модель, которая будет предсказывать риск водителя оказаться виновником ДТП. Заказчик исследования - каршеринговая компания, модель планируется использовать в бизнесе.

### Исходные данные
Исходные данные представляют собой SQL базу данных из трех таблиц. Данные содержат ошибки, пропуски, однако датасет большой, в главной таблице 1_400_000 записей, соответствующих различным ДТП. Поля для связи таблиц есть, данные можно успешно объединить. В целом датасет пригоден для дальнейшей работы. 

### Статистический анализ факторов ДТП
Был проведен статистический анализ факторов ДТП. Была проверена зависимость количества ДТП от времени года (явная - отсутствует), выявлены наиболее типичные обстоятельства самых серьезных ДТП. Кроме того, были поставлены дополнительные задачи для дальнейшего исследования.

Также была выявлена неполнота исходных данных: данные за 2012 год условно полные только за 5 месяцев, начиная с июня данные неполные или отсутствуют.

### Формирование датасета для обучения моделей
На основании исходной базы данных был сформирован датасет для обучения моделей. Для этого были оставлены:
* все данные начиная с 2012 года (как более свежие)
* все происшествия, в которых стороной ДТП является водитель машины
* все происшествия с значимыми последствиями (кроме типа 'scratch')

### Предобработка данных
В ходе предобработки данных были удалены лишние колонки `case_id`, `id`, `party_number` (служебные колонки), `county_city_location`, `county_location` (поскольку данные из одного региона, а задача не выявить зависимость риска от локации внутри региона, а экстраполировать данные региона на любой другой регион), `direction` (направление движения предположительно имеет значение только внутри заданного региона). 

Колонки `collision_date` и `collision_time` были удалены. Данные из колонки `collision_time` были перенесены в колонку `time` (категориальные значения: утро, день, ночь, вечер) на этапе выгрузки данных. 

В датасете были для удобства переименованы некоторые колонки, изменен тип данных для некоторых колонок, заполнены пропуски, устранены выбросы (в колонке `vehicle_age`). В результате в категориальных колонках осталось 95 уникальных значений.

В готовом к обучению моделей датасете 195_898 строк и 23 колонки.

### Исследование корреляций в исходном датасете
Корреляции целевого признака с обучающими в исходном датасете слабые. Наиболее сильные две обратные корреляции (рассчитаны по Спирмену); виновность водителя коррелирует с:
* `party_count` - количеством сторон-участников ДТП (К = -0.28)
* `insurance_premium` - суммой страховки (К = -0.13)

### Обучение моделей
На датасете обучено три вида моделей: линейная регрессия, CatBoost и LightGBM. Гиперпараметры для CatBoost и LightGBM подобраны на RandomizedSearchCV, для линейной регрессии - на GridSearchCV. 

В качестве ключевой метрики выбрана F1 score - среднее гармоническое полноты и точности.

Метрики моделей на валидации:



In [ ]:
results

### Выбор лучшей модели
Лучшая из получившихся моделей - catboost_model_4, обученная с настройками, подобранными на RandomizedSearchCV. Это разработка компании Яндекс.
Параметры модели:

In [ ]:
best_model.get_params()

Метрики модели на валидационных данных:

* F1 : 0.7431756663761068
* AUC_ROC : 0.7105323707217235
* Best threshold: 0.4

При запуске модели на тестовых данных метрика получилась примерно такая же:

* F1 : 0.7388942595405791
* AUC_ROC : 0.7071206946239839
* Best threshold: 0.4

Другие модели на тесте дают похожий результат.

### Анализ основных факторов на основании анализа весов модели

Выделение моделью важнейших факторов представляет особый интерес при условии слабой корреляции целевого признака с обучающими, которую мы выявили на этапе оценки датасета.

Поскольку модель CatBoost обучалась на датасете, кодированном методом OHE, то в качестве важных она выделила отдельные варианты перечисленных ниже факторов.

Модель определила как наиболее важные следующие факторы: `primary_collision_factor`, `party_count`, `pcf_violation_category`, `party_sobriety`. Важными также показались `motor_vehicle_incolved_with` и `insurance_premium`. Все это факторы, описывающие поведение водителя, а также взаимодействие водителя с другими участниками ДТП.

Неважными для модели оказались конструктивные особенности автомобиля и его состояние, погода, освещенность и состояние дороги, а также тип местности, где произошло ДТП.


#### Сопоставление с результатами модели LGB
Модель LGB определила как наиболее важные следующие факторы: 'insurance_premium', 'pcf_violation_category', 'distance' (расстояние от главное дороги). Менее важными показались факторы: 'party_count', 'motor_vehicle_involved_with', 'driver_condition'.

Неважными для модели оказались конструктивные особенности автомобиля и его состояние, погода, освещенность и состояние дороги, время суток, время года, а также тип местности, где произошло ДТП.

### Предложения по дальнейшей доработке модели
Имеет смысл обучить модель на всем датасете, не только на данных за 2012 год, и посмотреть результат.

Обучить модель, предсказывающую риск того, что водитель станет виновником ДТП, на этих данных можно, но стоит дообучить модель, чтобы повысить полноту предсказаний.